In [1]:
from osgeo import ogr
from osgeo import gdal
import ipyparallel as p

In [2]:
fl = './western_us_vic_index.tiff'
ds = gdal.Open(fl)
rast = np.array(ds.GetRasterBand(1).ReadAsArray())
rast[rast==-9999] = np.NaN

In [3]:
cells = np.unique(rast[np.isnan(rast)==0])

In [4]:
cells

array([  13120.,   13121.,   13418., ...,  252052.,  252053.,  252054.])

In [5]:
len(cells)

101709

In [6]:
pc = p.Client()

In [15]:
v = pc.load_balanced_view()

In [16]:
pc.ids

[0, 1, 2, 3]

In [35]:
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']
dates = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/timecode.pcl')

In [9]:
%%px
import numpy as np
import pandas as pd
files = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/forcing_fluxes_filenames_lat_lon_index.df')
dates = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/timecode.pcl')
wyears = np.load('/Volumes/Users/Theo/projects/Budyko_vic/wyears.npy')
fluxes_columns = ['y','m','d','ET','R','BF','sm1','sm2','sm3','SWE','Cs','Qs','Ql','Qg','NR','PEText','PETtrc','PETsrc']

In [11]:
files = pd.read_pickle('/Volumes/Users/Theo/projects/Budyko_vic/forcing_fluxes_filenames_lat_lon_index.df')

In [12]:
fl = files.loc[files['indexer']==387,'flux'].as_matrix()

In [13]:
fl[0]

'/Users/barnhatb/research/vic/fluxes/ascii/latitude.25.09375/flux_snow_25.09375_-99.46875'

In [56]:
def extractPET(idx):
    
    idx = int(idx)
    
    fl = files.loc[files['indexer']==idx,'flux'].as_matrix()[0]
    
    lat = float(fl.split('_')[-2]) # extract latitude from file name
    lon = float(fl.split('_')[-1]) # extract longitude from file name

    flux = pd.read_table(fl, sep='\t', names=fluxes_columns)

    flux.index = pd.DatetimeIndex(dates)

    pet = flux.loc['2000-01-01':'2013-12-31','PEText'].mean()
    
    return idx,lat,lon,pet

In [79]:
res = v.map(extractPET,cells)

In [86]:
res.progress/float(len(cells))*100.

100.0

In [81]:
len(cells)

101709

In [87]:
idx,lat,lon,pet = zip(*res.result)

In [88]:
np.savez_compressed('./vic_wUS_PET.npz',idx=idx,lat=lat,lon=lon,pet=pet)

In [90]:
ls -ltrh

total 1314886
-rwx------  1 barnhatb  staff   1.4M Dec 18 08:45 TRMM_Readme_v3.pdf*
-rwx------@ 1 barnhatb  staff   1.7M Jan 14 11:43 myfile.dat*
-rwx------@ 1 barnhatb  staff   336B Jan 15 10:49 get_smap_l3sm*
drwx------  1 barnhatb  staff    16K Jan 15 11:13 SMAP_L3SM/
-rwx------@ 1 barnhatb  staff   1.5K Jan 15 17:54 get_mod16A2_monthly*
-rwx------@ 1 barnhatb  staff   108B Jan 16 19:02 westernUS.shx*
-rwx------@ 1 barnhatb  staff    17K Jan 16 19:02 westernUS.shp.xml*
-rwx------@ 1 barnhatb  staff    48K Jan 16 19:02 westernUS.shp*
-rwx------@ 1 barnhatb  staff   116B Jan 16 19:02 westernUS.sbx*
-rwx------@ 1 barnhatb  staff   132B Jan 16 19:02 westernUS.sbn*
-rwx------@ 1 barnhatb  staff   145B Jan 16 19:02 westernUS.prj*
-rwx------@ 1 barnhatb  staff   242B Jan 16 19:02 westernUS.dbf*
-rwx------@ 1 barnhatb  staff     5B Jan 16 19:02 westernUS.cpg*
-rwx------  1 barnhatb  staff    50K Jan 17 16:51 Plot NC precip.ipynb*
-rwx------  1 barnhatb  staff   6.3K Jan 17 18:16 test.tiff.a